In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import joblib
seed = 11

In [2]:
# Считываем данные
np.random.seed(seed)
df = sns.load_dataset('diamonds').drop(columns=["color", "clarity", "depth", "table"])
df = df.sample(df.shape[0], ignore_index=True)
df

,carat,cut,price,x,y,z
0,1.51,Fair,5750,7.16,7.12,4.65
1,0.31,Premium,698,4.40,4.38,2.63
2,0.33,Ideal,427,4.44,4.46,2.77
3,0.28,Ideal,625,4.24,4.20,2.59
4,0.34,Ideal,765,4.50,4.48,2.74
...,...,...,...,...,...,...
53935,0.33,Very Good,781,4.43,4.46,2.74
53936,0.90,Ideal,4198,6.24,6.18,3.82
53937,1.45,Very Good,9683,7.32,7.41,4.46
53938,0.40,Ideal,945,4.70,4.68,2.95


In [3]:
# Смотрим информацию: тип данных и количество пропусков
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   carat   53940 non-null  float64 
 1   cut     53940 non-null  category
 2   price   53940 non-null  int64   
 3   x       53940 non-null  float64 
 4   y       53940 non-null  float64 
 5   z       53940 non-null  float64 
dtypes: category(1), float64(4), int64(1)
memory usage: 2.1 MB


In [4]:
# Смотрим количество пропусков в датасете
df.isna().sum()

carat    0
cut      0
price    0
x        0
y        0
z        0
dtype: int64

In [5]:
# Наблюдаем количество дубликатов
df.duplicated().sum()

457

In [6]:
# Удаляем дубликаты
df = df.drop_duplicates(ignore_index=True)
df.shape

(53483, 6)

In [7]:
# Удалим строки с 0 значениями
zero_vals = df[(df == 0).max(axis=1)].index
df.drop(zero_vals, inplace=True)

C:\Users\Andrew\AppData\Local\Temp\ipykernel_25700\1676052500.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(zero_vals, inplace=True)


In [8]:
# посмотрим на то, какие у нас есть категориальные и численные столбцы

cat_columns = []  # создаем пустой список для имен колонок категориальных данных
num_columns = []  # создаем пустой список для имен колонок числовых данных

for column_name in df.columns:  # смотрим на все колонки в датафрейме
    if (df[column_name].dtypes == 'category'):  # проверяем тип данных для каждой колонки
        cat_columns += [column_name]  # если тип объект - то складываем в категориальные данные
    else:
        num_columns += [column_name]  # иначе - числовые

# важно: если признак категориальный, но хранится в формате числовых данных, тогда код не сработает корректно

# выводим результат
print('Категориальные данные:\t ',cat_columns, '\n Число столблцов = ',len(cat_columns))

print('Числовые данные:\t ',  num_columns, '\n Число столблцов = ',len(num_columns))

Категориальные данные:	  ['cut'] 
 Число столблцов =  1
Числовые данные:	  ['carat', 'price', 'x', 'y', 'z'] 
 Число столблцов =  5


## Создание датасетов

In [9]:
np.random.seed(seed)
step = df.shape[0] // 6  # возьмём шаг в 1/6 датасета
# для train возьмём половину датасета
# для остальных возьмём по 1/6 датасета
df_train = df.iloc[0:3*step]
df_val = df.iloc[3*step:4*step]
df_test = df.iloc[4*step:5*step]
df_noise = df.iloc[5*step:df.shape[0] + 1]
df_noise.loc[:, 'price'] = df_noise['price'] + 1000 * np.random.random(df_noise.shape[0])

df_train.to_csv('./datasets/df_train.csv', index=False)
df_val.to_csv('./datasets/df_val.csv', index=False)
df_test.to_csv('./datasets/df_test.csv', index=False)
df_noise.to_csv('./datasets/df_noise.csv', index=False)

In [10]:
# Проверим, что всё взято верно
df_train.shape[0] + \
df_val.shape[0] + \
df_test.shape[0] + \
df_noise.shape[0] == df.shape[0]

True

## Препроцессинг и обучение модели

In [11]:
# Создание пайплайна для препроцессинга данных и выполнения прогноза
transformer = ColumnTransformer([
    ('OHE', OneHotEncoder(), ['cut']),
    ('std_scale', StandardScaler(), ['carat', 'x', 'y', 'z']),
])

pipe = Pipeline([
    ('preprocessing', transformer),
    ('model', LinearRegression())
])

In [12]:
X_train = df_train.drop(columns=['price'])
y_train = df_train['price']

In [13]:
# Обучение модели
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('OHE', OneHotEncoder(),
                                                  ['cut']),
                                                 ('std_scale', StandardScaler(),
                                                  ['carat', 'x', 'y', 'z'])])),
                ('model', LinearRegression())])

In [14]:
pipe['preprocessing'].get_feature_names_out()

array(['OHE__cut_Fair', 'OHE__cut_Good', 'OHE__cut_Ideal',
       'OHE__cut_Premium', 'OHE__cut_Very Good', 'std_scale__carat',
       'std_scale__x', 'std_scale__y', 'std_scale__z'], dtype=object)

In [15]:
pipe['model'].coef_

array([-3.17195120e+13, -3.17195120e+13, -3.17195120e+13, -3.17195120e+13,
       -3.17195120e+13,  5.41125391e+03, -5.88492188e+02,  1.70675781e+02,
       -1.29229688e+03])

In [16]:
# Сохранение обученной модели
joblib.dump(pipe, "./models/pipe_model.joblib")

['./models/pipe_model.joblib']

## Создание файла с тестами

Создадим файл с конфигурацией для pytest и реализуем доп. параметр --dataset для передачи датасета из коммандной строки.

In [17]:
%%writefile conftest.py
def pytest_addoption(parser):
    parser.addoption(
        "--dataset",
        default=[],
        type=str,
        action='append',
        help="path to dataset to test model",
    )

def pytest_generate_tests(metafunc):
    metafunc.parametrize("dataset", metafunc.config.getoption("dataset"))

Overwriting conftest.py


In [18]:
%%writefile test.py
import pandas as pd
import pytest
import joblib
from sklearn.metrics import mean_absolute_error, r2_score


@pytest.fixture()
def load_model():
    model = joblib.load('./models/pipe_model.joblib')
    return model


@pytest.fixture()
def load_dataset(dataset):
    df = pd.read_csv(dataset)
    return df


@pytest.fixture()
def make_prediction(load_model, load_dataset):
    X = load_dataset.drop(columns='price')
    y = load_dataset['price']
    y_pred = load_model.predict(X)
    return y, y_pred


def test_MAE(make_prediction):
    y, y_pred = make_prediction
    assert mean_absolute_error(y, y_pred) < 900


def test_R2(make_prediction):
    y, y_pred = make_prediction
    assert r2_score(y, y_pred) > 0.85


Overwriting test.py


## Тестирование датасетов

In [19]:
!pytest -v test.py --dataset=./datasets/df_train.csv

============================= test session starts =============================
platform win32 -- Python 3.11.4, pytest-8.2.2, pluggy-1.5.0 -- C:\Users\Andrew\AppData\Local\Programs\Python\Python311\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\Andrew\Desktop\UrFU_MLOps_course\lab5
plugins: anyio-4.2.0, hydra-core-1.3.2
collecting ... collected 2 items

test.py::test_MAE[./datasets/df_train.csv] PASSED                        [ 50%]
test.py::test_R2[./datasets/df_train.csv] PASSED                         [100%]

============================== 2 passed in 1.16s ==============================


In [20]:
!pytest -v test.py --dataset=./datasets/df_val.csv

============================= test session starts =============================
platform win32 -- Python 3.11.4, pytest-8.2.2, pluggy-1.5.0 -- C:\Users\Andrew\AppData\Local\Programs\Python\Python311\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\Andrew\Desktop\UrFU_MLOps_course\lab5
plugins: anyio-4.2.0, hydra-core-1.3.2
collecting ... collected 2 items

test.py::test_MAE[./datasets/df_val.csv] PASSED                          [ 50%]
test.py::test_R2[./datasets/df_val.csv] PASSED                           [100%]

============================== 2 passed in 1.18s ==============================


In [21]:
!pytest -v test.py --dataset=./datasets/df_test.csv

============================= test session starts =============================
platform win32 -- Python 3.11.4, pytest-8.2.2, pluggy-1.5.0 -- C:\Users\Andrew\AppData\Local\Programs\Python\Python311\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\Andrew\Desktop\UrFU_MLOps_course\lab5
plugins: anyio-4.2.0, hydra-core-1.3.2
collecting ... collected 2 items

test.py::test_MAE[./datasets/df_test.csv] PASSED                         [ 50%]
test.py::test_R2[./datasets/df_test.csv] PASSED                          [100%]

============================== 2 passed in 1.17s ==============================


In [22]:
!pytest -v test.py --dataset=./datasets/df_noise.csv

============================= test session starts =============================
platform win32 -- Python 3.11.4, pytest-8.2.2, pluggy-1.5.0 -- C:\Users\Andrew\AppData\Local\Programs\Python\Python311\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\Andrew\Desktop\UrFU_MLOps_course\lab5
plugins: anyio-4.2.0, hydra-core-1.3.2
collecting ... collected 2 items

test.py::test_MAE[./datasets/df_noise.csv] FAILED                        [ 50%]
test.py::test_R2[./datasets/df_noise.csv] FAILED                         [100%]

================================== FAILURES ===================================
______________________ test_MAE[./datasets/df_noise.csv] ______________________

make_prediction = (0        1470.269689
1        1624.475241
2        1243.218526
3        7538.933929
4       15582.203605
            ...float64, array([1109.4765625 , 2207.51953125,  770.18359375, ..., 9090.74609375,
       1137.5234375 , 4927.4609375 ]))

    def test_MAE(make_prediction):
        y, y_pred = 

Видно, что зашумлённый датасет не проходит проверку.